## Data Creation: The 1st Approach in Link Prediction
In this Jupyter notebook, we embark on the journey of creating a dataset for link prediction in a graph and extracting various similarity metrics to better understand the relationships between nodes. This dataset will consist of potential edges, each labeled as follows:

- Label `1` signifies that the edge represents an actual connection.
- Label `0` indicates that the edge does not exist.

We have access to information regarding the real edges, labeled as `1`. These real edges are divided into two categories: `Eprev` for training and `Enext` for testing. The challenge at hand is to construct non-existing edges (labeled as `0`) to create a well-balanced dataset. These non-existing edges belong to a set known as `Enon`, which is substantially larger in size compared to both `Eprev` and `Enext`.

     The Approach
The objective is to maintain this balance by implementing various strategies to sample non-existing edges, and subsequently comparing the effectiveness of these strategies. However, in this jyputer nootbook we will apply the first approach, which involves **`connecting less popular nodes to neighbors of more popular nodes`**. This strategy aims to create a dataset where non-existing edges exhibit similarities in terms of distance and Preferential Attachment to existing edges. Still, we will explore more alternative methods in the other notebooks to identify the most promising approach.

Now, we embark on dataset creation, meticulously crafting the foundation for our link prediction. The subsequent notebook, titled 'Link_Prediction_of_1st_Approach', will contain a comprehensive evaluation of our dataset and the performance of this specific sampling strategy for the link prediction project.

In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import math
from math import log
import random
import csv

import warnings
warnings.filterwarnings('ignore')

In [2]:
output_file = '../Dataset/stackoverflow_Graphs.csv'
df = pd.read_csv(output_file)
len(df)

63497050

### Check Monotonic Increasing Timestamp Order

In [3]:
df.Timestamp.is_monotonic_increasing

False

In [4]:
df.sort_values(by= 'Timestamp',inplace= True)
df.Timestamp.is_monotonic_increasing

True

### Define Time Periods for Data Segmentation

In [5]:
# Define the N parameter for monthly intervals
N = int(2774 / 30 )
# Calculate the set of time instances
# -----------------------------------
# Step 1: Calculate the time interval length
DT =  df.Timestamp.max() - df.Timestamp.min()
# Step 2: Calculate the length of each non-overlapping time period
dt = DT / N
# Step 3 : Determine the set of time instances {t0, ..., tN}
time_instances = [df.Timestamp.min() + ( j * dt) for j in range(N+1)]

# Calculate the set of non-overlapping time periods 
# -------------------------------------------------
# The time_instances list will contain N+1 time instances which represent the start times of each non-overlapping time period {T1, ..., TN}
# The end times of each time period will be the following time instance in the time_instances list
# Determine the set of non-overlapping time periods {T1, ..., TN}
time_periods = [(time_instances[i], time_instances[i + 1]) for i in range(N)]

### Graph Creation and Subgraph Generation

In [6]:
def create_subgraph(subdf):
    subgraph = nx.from_pandas_edgelist(subdf, 'Source_Node', 'Target_Node')
    return subgraph

def subgraphs_generator(df, time_periods):
    for start, end in time_periods[:-1]:
        subdf = df[(df.Timestamp >= start) & (df.Timestamp < end)]
        subdf = subdf.drop_duplicates(subset=['Source_Node', 'Target_Node'])
        subdf = subdf[subdf['Source_Node'] != subdf['Target_Node']]
        subgraph = create_subgraph(subdf)
        yield subgraph

    start, end = time_periods[-1]
    subdf = df[(df.Timestamp >= start) & (df.Timestamp <= end)]
    subdf = subdf.drop_duplicates(subset=['Source_Node', 'Target_Node'])
    subdf = subdf[subdf['Source_Node'] != subdf['Target_Node']]
    subgraph = create_subgraph(subdf)
    yield subgraph


def ith_subgraph(df, time_periods, index):
    if index < len(time_periods) - 1:
        start, end = time_periods[index]
        subdf = df[(df.Timestamp >= start) & (df.Timestamp < end)]
        subdf = subdf.drop_duplicates(subset=['Source_Node', 'Target_Node'])
        subdf = subdf[subdf['Source_Node'] != subdf['Target_Node']]
        subgraph = create_subgraph(subdf)
        return subgraph
    else:
        start, end = time_periods[-1]
        subdf = df[(df.Timestamp >= start) & (df.Timestamp <= end)]
        subdf = subdf.drop_duplicates(subset=['Source_Node', 'Target_Node'])
        subdf = subdf[subdf['Source_Node'] != subdf['Target_Node']]
        subgraph = create_subgraph(subdf)
        return subgraph

### Creating Common Nodes and Their Connections for Link Prediction

In [7]:
# Function to find the common set of nodes between two subgraphs
def common_nodes(subgraph_current, subgraph_next):
    return set(subgraph_current.nodes()).intersection(subgraph_next.nodes())

# Function to find the restricted set of edges within the common set of nodes
def restricted_edges(subgraph, common_nodes):
    return [(u, v) for u, v in subgraph.edges() if (u in common_nodes and v in common_nodes)]

# Lists to store the sets for each pair of successive time periods
V_STAR = []
E_STAR_PREV = []
E_STAR_NEXT = []

# Initialize the first subgraph as the previous / current
graph_previous = ith_subgraph(df,time_periods,0)

# Loop through each pair of successive time periods (Tj, Tj+1) where 1 ≤ j ≤ N - 1, 
# starting from the second time period because the first one is already initialized.
for graph_next in subgraphs_generator(df, time_periods[1:]):

    # Calculate the common set of nodes V∗[tj−1, tj+1]
    v_star = common_nodes(graph_previous,graph_next)
    V_STAR.append(len(v_star))

    # Calculate the restricted sets of edges E∗[tj−1, tj] and E∗[tj, tj+1]
    e_star_prev = restricted_edges(graph_previous, v_star)
    e_star_next = restricted_edges(graph_next, v_star)
    E_STAR_PREV.append(len(e_star_prev))
    E_STAR_NEXT.append(len(e_star_next))

    # Update the previous graph for the next iteration
    graph_previous = graph_next

**Is important to check if we have an empty set in V***

In [10]:
V_STAR[0]

1713

### Create Functions To Calculating Similarity Metrics for Link Prediction

In [8]:
# Calculate the graph distance (SGD) between two nodes u and v in a subgraph
def graph_distance(subgraph, u, v):
    if nx.has_path(subgraph,u,v):
        shortest_path = nx.shortest_path_length(subgraph, source=u, target=v)
    else:
        shortest_path = math.inf # If there is no path, return 0 to represent maximum distance (disconnected nodes)
    return 1 / shortest_path

# Calculate the common neighbors (SCN) between two nodes u and v in a subgraph
def common_neighbors(subgraph, u, v):
    neighbors_u = set(subgraph[u])
    neighbors_v = set(subgraph[v])
    return len(neighbors_u.intersection(neighbors_v))

# Calculate Jaccard's coefficient (SJC) between two nodes u and v in a subgraph
def jaccard_coefficient(subgraph, u, v):
    neighbors_u = set(subgraph[u])
    neighbors_v = set(subgraph[v])
    union_size = len(neighbors_u.union(neighbors_v))
    if union_size == 0:
        return 0
    return len(neighbors_u.intersection(neighbors_v)) / union_size

# Calculate Adamic/Adar (SA) score between two nodes u and v in a subgraph
def adamic_adar(subgraph, u, v):
    common_neighbors = set(subgraph[u]).intersection(set(subgraph[v]))
    if len(common_neighbors) == 0:
        return 0
    return sum(1 / log(len(subgraph[neighbor]), 2) if log(len(subgraph[neighbor]), 2) != 0 else 0 for neighbor in common_neighbors)

# Calculate Preferential Attachment (SPA) between two nodes u and v in a subgraph
def preferential_attachment(subgraph, u, v):
    degree_u = subgraph.degree(u)
    degree_v = subgraph.degree(v)
    return degree_u * degree_v

### Generating Metrics for Existing and Non-Existing Edges Based On the Approach

In this section, we introduce our first approach for link prediction, which focuses on generating metrics for both existing and non-existing edges. The core idea is to calculate various similarity metrics that can be used to distinguish between actual connections (label `1`) and non-existing edges (label `0`). These metrics provide insights into the likelihood of a link between two nodes.

    Metrics for Existing Edges (label 1)
For existing edges (label `1`), we compute the following metrics to understand the similarities between connected nodes:
- **Graph Distance (SGD):** Measures the distance between nodes in the subgraph.
- **Common Neighbors (SCN):** Counts the number of shared neighbors between two nodes.
- **Jaccard's Coefficient (SJC):** Measures the overlap between neighbor sets of two nodes.
- **Adamic/Adar Score (SA):** Assigns a score based on shared neighbors, giving more weight to rarer neighbors.
- **Preferential Attachment (SPA):** Computes the product of the number of neighbors of two nodes.

    The Approach for Non-Existing Edges (label 0)
Next, we delve into handling non-existing edges (label `0`). Our approach involves connecting a less popular node to the neighbors of a more popular node. By doing so, we aim to create non-existing edges that exhibit similarities in terms of distance and Preferential Attachment to existing edges. 

We implement this approach, iterating through the graph to identify potential non-existing edges and calculate their metrics. If the approach fails to find suitable neighbors for a less popular node within a certain limit (pos=5), a random node from the graph is chosen to create a non-existing edge. This method maintains the balance between existing and non-existing edges in our dataset, providing a foundation for further link prediction analysis.

Please note that this is the first approach, and we will explore additional strategies in subsequent sections to identify the most promising method for link prediction.

In [11]:
#----------------
# Approach 1
#----------------
def get_metrics_gen(graph_star,e_star):
    # Initialize a set to keep track of non-existing edges
    e_non_ex = set()

    # Iterate through each potential edge in the list of existing edges (e_star)
    for (u,v) in list(e_star):

        # Calculate and yield Metrics for Existing Edges (label 1)
        dist = graph_distance(graph_star, u, v)
        neighbours = common_neighbors(graph_star, u, v)
        jaccard = jaccard_coefficient(graph_star, u, v)
        adar = adamic_adar(graph_star, u, v)
        pref = preferential_attachment(graph_star, u, v)
        label = 1
        yield (u,v),dist,neighbours,jaccard,adar,pref,label

    # Implementing Approach for Non-Existing Edges (label 0)

        # Get the degree of nodes u and v
        u_degree = graph_star.degree(u)
        v_degree = graph_star.degree(v)
        
        #Keep as source the less popular node
        if u_degree <= v_degree:
            source = u
            target = v
        else:
            source = v
            target = u

        # Find the neighbors of the source and target nodes
        source_neighbors = set(graph_star.neighbors(source))
        target_neighbors = set(graph_star.neighbors(target))

        # Find nodes not connected with the source
        probable_future_connection_with_source = target_neighbors - target_neighbors.intersection(source_neighbors)
        
        # Identify nodes that are not connected with the source node
        not_connected_with_source = [node for node in probable_future_connection_with_source if not graph_star.has_edge(node, source)]

        pos = 0
        # Attempt to find a suitable non-existing edge within a limit (pos=5)
        while True:
            if len(not_connected_with_source) > 0:
                v_new = random.choice(not_connected_with_source)
                if (source,v_new) not in e_star and (v_new,source) not in e_star and (source,v_new) not in e_non_ex and (v_new,source) not in e_non_ex and (source != v_new ):
                    #print('not Random')
                    break
            if pos >= 5 or len(not_connected_with_source) == 0:
                v_new = random.choice(list(graph_star.nodes))
                if (source,v_new) not in e_star and (v_new,source) not in e_star and (source,v_new) not in e_non_ex and (v_new,source) not in e_non_ex and (source != v_new ):
                    #print('random')
                    break
            pos += 1

        # Add the non-existing edge to the set
        e_non_ex.add((source,v_new))

        # Calculate nad yield Metrics for Non-Existing Edges (label 0)
        dist = graph_distance(graph_star, source, v_new)
        neighbours = common_neighbors(graph_star, source, v_new)
        jaccard = jaccard_coefficient(graph_star, source, v_new)
        adar = adamic_adar(graph_star, source, v_new)
        pref = preferential_attachment(graph_star, source, v_new)
        label = 0
        yield  (source,v_new) , dist, neighbours, jaccard, adar, pref, label

### Exporting Metrics for Training and Testing Datasets Based on Time Periods

In [ ]:
# Initialize lists to store metrics for training data
possible_edges_train = []
distance_train = []
neighbors_train = []
jaccard_train = []
adar_train = []
pref_train = []
label_train = []

# Initialize lists to store metrics for testing data
possible_edges_test = []
distance_test = []
neighbors_test = []
jaccard_test = []
adar_test = []
pref_test = []
label_test = []
c = 0

# Initialize the previous graph as the first subgraph
graph_previous = ith_subgraph(df,time_periods,0)

# Iterate through each pair of successive time periods (Tj, Tj+1)
for graph_next in subgraphs_generator(df, time_periods[1:]):
        c +=1
        #print(c) # Just count the iteration

        # Calculate the common set of nodes (V*) between the previous and next subgraphs
        v_star = common_nodes(graph_previous,graph_next)

        # Calculate the restricted sets of edges E*[tj−1, tj] and E*[tj, tj+1]
        e_star_prev = restricted_edges(graph_previous, v_star)
        e_star_next = restricted_edges(graph_next, v_star)

        # Create a graph (G_train) for the training data
        G_train = nx.Graph()
        G_train.add_nodes_from(v_star)
        G_train.add_edges_from(e_star_prev)

        # Create a graph (G_test) for the testing data
        G_test = nx.Graph()
        G_test.add_nodes_from(v_star)
        G_test.add_edges_from(e_star_next)

        # Calculate metrics for the training data
        for possible_edge, sgd, scn, sjc, sa, spa,label  in get_metrics_gen(G_train,e_star_prev):
                possible_edges_train.append(possible_edge)
                distance_train.append(sgd)
                neighbors_train.append(scn)
                jaccard_train.append(sjc)
                adar_train.append(sa)
                pref_train.append(spa)
                label_train.append(label)

        # Calculate metrics for the testing data
        for possible_edge, sgd, scn, sjc, sa, spa,label  in get_metrics_gen(G_test,e_star_next):
                possible_edges_test.append(possible_edge)
                distance_test.append(sgd)
                neighbors_test.append(scn)
                jaccard_test.append(sjc)
                adar_test.append(sa)
                pref_test.append(spa)
                label_test.append(label)

        # Update the previous graph for the next iteration
        graph_previous = graph_next

### Exporting Training Metrics to CSV File

In [ ]:
# Define the CSV file path for traing data
csv_file_path = "../Dataset/monthly_1st_Approache_trainData.csv"
chunksize = 10000 # Define the chunk size for writing data

with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['possible_edges', 'distance', 'neighbors', 'jaccard', 'adar', 'pref','label'])  # Write header
    
    chunk = [] # Initialize an empty chunk to store data temporarily
    for row in zip(possible_edges_train, distance_train, neighbors_train, jaccard_train, adar_train, pref_train,label_train):
        chunk.append(row) # Add the current row to the chunk
        
        if len(chunk) == chunksize:
            writer.writerows(chunk) # Write the chunk to the CSV file
            chunk = [] # Clear the chunk for the next set of data
    
    if chunk:
        writer.writerows(chunk) # Write the remaining data in the chunk to the CSV file

### Exporting Testing Metrics to CSV File

In [ ]:
# Define the CSV file path for testing data
csv_file_path = "../Dataset/monthly_1st_Approache_testData.csv"
chunksize = 10000

# Create a CSV file for testing data
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['possible_edges', 'distance', 'neighbors', 'jaccard', 'adar', 'pref','label'])  # Write header
    
    chunk = []
    for row in zip(possible_edges_test, distance_test, neighbors_test, jaccard_test, adar_test, pref_test,label_test):
        chunk.append(row)
        
        if len(chunk) == chunksize:
            writer.writerows(chunk)
            chunk = []
    
    if chunk:
        writer.writerows(chunk) 